In [ ]:
#We will look at a different apporach quantum channels using quantum circuits

#Using Quantum Communication

In [1]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer

In [6]:
qreg = QuantumRegister(8)
creg = ClassicalRegister(8)


#Quantum Circuit for Asja state
asja = QuantumCircuit(qreg, creg, name = 'Asja')

#Now we will randomly choose qubits from 0 to 7 to apply a X gate

from random import randrange

send = []

#We selected according to what random pattern we need to send 
for i in range(8):
    bit = randrange(2)
    send.append(bit)

#Now applying X gates
for i, n in enumerate(send):
    if n == 1:
        asja.x(qreg[i])

send_str = ''.join(str(e) for e in send)
print(send_str)

asja.draw()


10010011


┌───┐
q4_0: ┤ X ├
      └───┘
q4_1: ─────
           
q4_2: ─────
      ┌───┐
q4_3: ┤ X ├
      └───┘
q4_4: ─────
           
q4_5: ─────
      ┌───┐
q4_6: ┤ X ├
      ├───┤
q4_7: ┤ X ├
      └───┘
c4: 8/═════

In [7]:
#Now how we will send these states to Balvis?

# import all necessary objects and methods for quantum circuits
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from random import randrange

#Source: awards/teach_me_qiskit_2018/cryptography/Cryptography.ipynb

def SendState(qc1, qc2, qc1_name):
    ''' This function takes the output of a circuit qc1 (made up only of x and 
        h gates and initializes another circuit qc2 with the same state
    ''' 
    
    # Quantum state is retrieved from qasm code of qc1
    qs = qc1.qasm().split(sep=';')[4:-1]

    # Process the code to get the instructions
    for index, instruction in enumerate(qs):
        qs[index] = instruction.lstrip()

    for instruction in qs:
        if instruction[0] == 'x':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.x(qreg[old_qr])
        elif instruction[0] == 'h':
            if instruction[5] == '[':
                old_qr = int(instruction[6:-1])
            else:
                old_qr = int(instruction[5:-1])
            qc2.h(qreg[old_qr])
        elif instruction[0] == 'm': # exclude measuring:
            pass
        else:
            raise Exception('Unable to parse instruction')

In [8]:
balvis = QuantumCircuit(qreg, creg, name='Balvis') #Defining Balvis circuit

SendState(asja, balvis, 'Asja') #Asja sends states to Balvis

balvis.draw()

C:\Users\jrnat\AppData\Local\Temp\ipykernel_17520\1419382280.py:15: DeprecationWarning: The method ``qiskit.circuit.quantumcircuit.QuantumCircuit.qasm()`` is deprecated as of qiskit 0.46. It will be removed in the 1.0.0 release. Instead use the qiskit.qasm2.dump() or qiskit.qasm2.dumps() function
  qs = qc1.qasm().split(sep=';')[4:-1]


┌───┐
q4_0: ┤ X ├
      └───┘
q4_1: ─────
           
q4_2: ─────
      ┌───┐
q4_3: ┤ X ├
      └───┘
q4_4: ─────
           
q4_5: ─────
      ┌───┐
q4_6: ┤ X ├
      ├───┤
q4_7: ┤ X ├
      └───┘
c4: 8/═════

In [9]:
#We just sent quantum states from Asja to Balvis

#Now let's see what happens Balvis performs measurement to read the bits Asja sent to him

balvis.measure(qreg,creg)

In [10]:
def print_outcomes_in_reserve(counts): # takes a dictionary variable
    for outcome in counts: # for each key-value in dictionary
        reverse_outcome = ''
        for i in outcome: # each string can be considered as a list of characters
            reverse_outcome = i + reverse_outcome # each new symbol comes before the old symbol(s)
    return reverse_outcome

job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1) #Note that Balvis only has one shot to measure qubits
counts = job.result().get_counts(balvis) # counts is a dictionary object in python
received = print_outcomes_in_reserve(counts)

C:\Users\jrnat\AppData\Local\Temp\ipykernel_17520\1171200962.py:8: DeprecationWarning: The 'qiskit.Aer' entry point is deprecated and will be removed in Qiskit 1.0. You should use 'qiskit_aer.Aer' directly instead.
  job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1) #Note that Balvis only has one shot to measure qubits
C:\Users\jrnat\AppData\Local\Temp\ipykernel_17520\1171200962.py:8: DeprecationWarning: The function ``qiskit.execute_function.execute()`` is deprecated as of qiskit 0.46.0. It will be removed in the Qiskit 1.0 release. This function combines ``transpile`` and ``backend.run``, which is covered by ``Sampler`` :mod:`~qiskit.primitives`. Alternatively, you can also run :func:`.transpile` followed by ``backend.run()``.
  job = execute(balvis,Aer.get_backend('qasm_simulator'),shots=1) #Note that Balvis only has one shot to measure qubits


In [11]:
print("Asja sent:", send_str)
print("Balvis received:", received)

Asja sent: 10010011
Balvis received: 10010011


In [ ]:
#Hence we can see the same set of bits have been communicated
